In [33]:

import os
import sys
import re
import gc
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer ,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline ,Pipeline
from sklearn.metrics import classification_report

from joblib import dump


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [34]:

BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroups')
# This code from http#s://www.kaggle.com/mansijharia with some edit
# May take a few time 

texts = []
labels_index = {}
labels = []

for name in sorted(os.listdir((BASE_DIR+'20_newsgroups'))):
    path = os.path.join(BASE_DIR,'20_newsgroups', name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    #Skip the matadata at 1st pragraph.
                    i = t.find('\n\n')
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

In [35]:
#cleaing by RE 
for i in range(0,len(texts)):
    texts[i] =texts[i].strip()
    texts[i] =re.sub(r'\=+',' ', texts[i])#To remove any == characters
    texts[i] =re.sub(r'\|+',' ', texts[i])#To remove any | characters
    texts[i] =re.sub('[<>]', ' ',texts[i])#To remove < and > characters
    texts[i] =re.sub(r'\[\]+',' ', texts[i])#To remove any [] characters
    texts[i] =re.sub(r'\(\)+',' ', texts[i])#To remove any () empty parentheses
    texts[i] =re.sub('--+', ' ',texts[i])#To remove multiple spaces 
    texts[i] =re.sub("[__]+", ' ', texts[i])#To remove lines
    texts[i] =re.sub('\^+',' ', texts[i])#To remove ^ characters  
    texts[i] =re.sub(r'[/*\\*/]',' ', texts[i])#To remove \/ characters
    texts[i] =re.sub('([\w\.-]+)@([\w\.-]+)',' ', texts[i])#To remove any emails 
    texts[i] =re.sub('[^a-zA-Z]',' ',texts[i])#To remove nunEnglish and digital characters
    texts[i] =re.sub(r'\~\~+',' ', texts[i])#To remove any == characters
    texts[i] =re.sub('\n', ' ', texts[i])#To remove lines
    texts[i] =re.sub('\t', ' ',texts[i])#To remove tab spaces 
    texts[i] =re.sub('  +', ' ',texts[i])#To remove multiple spaces 
    # if i==12:
    #     break

In [36]:
#here we need to reduce the number of words
def size_word(texts):
    short_words=[]
    for j in range(0,len(texts)):
        clean_tokens=tokanizer(texts[j])
        texts[j] =[]              
        for i in range(0,len(clean_tokens)):
            if (len(clean_tokens[i])>=3 | len(clean_tokens[i])<=15):
                texts[j].append(clean_tokens[i])
                return texts

In [37]:
def tokanizer(texts):
    clean_tokens=[]
    Lemmatizer=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))
    for i in range(0,len(texts)):
        tokens = word_tokenize(texts[i])
        for tokn in tokens:
            clean_tok= Lemmatizer.lemmatize(tokn) 
            if clean_tok not in stop_words:
                clean_tokens.append(clean_tok)
    return clean_tokens        

 

In [38]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels,test_size=0.33)
count_vect =  CountVectorizer()
transformer =  TfidfTransformer()
count_vect.fit(X_train)
count_tran = count_vect.transform(X_train)
transformer.fit(count_tran)
tf_transform = transformer.transform(count_tran)

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(tf_transform,y_train)
c_tran_test = count_vect.transform(X_test)
y_pred = classifier.predict(transformer.transform(c_tran_test))
print(classification_report(y_pred,y_test))
# dump(classifier, '\Group Project') 


              precision    recall  f1-score   support

           0       0.53      0.60      0.56       306
           1       0.68      0.62      0.65       389
           2       0.80      0.66      0.72       384
           3       0.69      0.67      0.68       335
           4       0.77      0.77      0.77       314
           5       0.78      0.77      0.78       327
           6       0.82      0.67      0.73       414
           7       0.80      0.79      0.79       325
           8       0.88      0.92      0.90       320
           9       0.89      0.80      0.84       354
          10       0.92      0.91      0.92       335
          11       0.81      0.88      0.85       306
          12       0.55      0.73      0.62       242
          13       0.80      0.85      0.82       325
          14       0.87      0.87      0.87       320
          15       0.86      0.67      0.75       426
          16       0.80      0.69      0.74       386
          17       0.87    

In [ ]:
n_neighbors = 3
weights = 'uniform'
weights = 'distance'
classifier =KNeighborsClassifier(n_neighbors, weights=weights)
classifier.fit(tf_transform,y_train)
c_tran_test = count_vect.transform(X_test)
y_pred = classifier.predict(transformer.transform(c_tran_test))
print(classification_report(y_pred,y_test))
# dump(classifier, '\Group Project')


              precision    recall  f1-score   support

           0       0.36      0.62      0.46       197
           1       0.31      0.90      0.47       123
           2       0.43      0.94      0.59       147
           3       0.36      0.95      0.53       126
           4       0.98      0.09      0.16      3567
           5       0.39      0.96      0.55       130
           6       0.22      0.90      0.35        81
           7       0.47      0.95      0.63       159
           8       0.42      0.99      0.59       141
           9       0.46      0.96      0.62       152
          10       0.51      0.98      0.67       171
          11       0.46      0.99      0.63       153
          12       0.43      0.95      0.59       146
          13       0.51      0.97      0.67       181
          14       0.44      0.99      0.61       142
          15       0.46      0.97      0.63       158
          16       0.51      0.86      0.64       200
          17       0.61    

In [39]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english', ngram_range = (1,2), binary = True, sublinear_tf=True)), # CountVectorizer, TfidfVectorizer
        ('clf', KNeighborsClassifier())])  # MultinomialNB, LogisticRegression, SGDClassifier

# parameters = {
#        'clf__n_neighbors': (5, 10, 100, 200),
#        'clf__weights': ('uniform', 'distance')
# }
Bast_parameters = {
       'clf__n_neighbors':(5,7) ,
       'clf__weights': ('distance','uniform')
}

grid_search = GridSearchCV(pipeline, Bast_parameters)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))
#16:43 m

C:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
C:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
C:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
C:\U

0.2633333333333333


In [ ]:
# estimators = [
#      ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42))),
#      ('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
# clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
# clf.fit(X_train, y_train).score(X_test, y_test)